In [ ]:
from twikit import Client
import time
import json
from twikit import TwitterException 
import pandas as pd
import sqlite3

# this API requires authentication
f = open('authentication.txt', 'r')
auth = f.read()
f.close()
auth_token = auth.split("\n")

# don't hardcode your email and password into something!!!
# the auth is in gitignore so I won't get hacked
USERNAME = str(auth_token[0])
EMAIL = str(auth_token[1])
PASSWORD = str(auth_token[2])

# Initialize client
client = Client(language='en-US', http2=True)

# Login to the service with provided user credentials
client.login(
    auth_info_1=USERNAME ,
    auth_info_2=EMAIL,
    password=PASSWORD
)

In [ ]:
# Twitter LOVES to ban people when they log in repeatedly
# saving the cookies makes sure I don't get banned (often)

client.get_cookies()
client.save_cookies('cookies.json')
with open('cookies.json', 'r', encoding='UTF8') as f:
    client.set_cookies(json.load(f))

In [ ]:
from twikit import UserUnavailable
from twikit import UserNotFound

def get_user_id(num, handle):
    # load the cookies so you don't login a million times and get banned
    client.load_cookies('cookies.json')
    
    # pull IDs and insert them back into the list
    try:
        user_id = client.get_user_by_screen_name(handle).id
        print(f'{num} Success {handle}, {user_id}')

    # if user is inaccessible these will keep the loop from breaking
    except UserUnavailable:
        print(f"{num} User {handle} unavailable")
    except UserNotFound:
        print(f'{num} User {handle} not found')

    return(user_id)

In [ ]:
def create_key(user_id, year):
    # load the cookies so you don't login a million times and get banned
    return(int(str(user_id) + str(year)))

In [ ]:
from math import ceil
import requests
from twikit import TooManyRequests
from twikit.utils import Endpoint

# this is a housekeeping function
# twitter API can throw rate limits
# they're kind of like timeouts
# this function just shows me how much longer I will be in timeout for

def get_limit_reset_time(endpoint: str):
    res = requests.get(
        endpoint,
        headers=client._base_headers,
        cookies=client.get_cookies()
    )
    return ceil(int(res.headers['x-rate-limit-reset']) - time.time())

In [ ]:
# timeout check for accessing user IDs
try:
    print(client.get_user_by_screen_name('BarackObama'))
except TooManyRequests:
    
    reset_time = get_limit_reset_time(Endpoint.USER_TWEETS)
    print(f'rate limit is reset after {reset_time} seconds.')

In [ ]:
data = pd.read_csv("data/twitter politicians.csv", keep_default_na=False)
df = pd.DataFrame(data)

# turn pandas into list
# there is probably a better way to do this but this works 
data_list = []
for index, row in df.iterrows():

    row_list = df.loc[index, :].values.flatten().tolist()
    data_list.append(row_list)
    
batch1 = [item for item in data_list[0:77]]
batch2 = [item for item in data_list[77:]]

mass_list = [batch1, batch2]

In [ ]:
i = 1
final_list = []

for lst in mass_list:
    # tracking which batch we are on
    print(i)
    j = 1
    for item in lst:
        user_id = get_user_id(j, item[2])
        item[0] = user_id
        final_list.append(item)
        time.sleep(1)
        j += 1

    if i == 1:
        time.sleep(900) # cooldown to avoid rate limits

    i += 1
    print("") 

In [ ]:
# read in coordinates data
data = pd.read_csv("data/coordinates.csv", keep_default_na=False)
df = pd.DataFrame(data)
data_list = []

# flatten it so SQL can ingest it
for index, row in df.iterrows():

    row_list = df.loc[index, :].values.flatten().tolist()
    row_list[1] = str(row_list[1])
    data_list.append(row_list[1:])

In [ ]:
# this ticker tracks the hits
i = 1

# this ticker tracks the batches
# we can really only hit this endpoint about 90 times per every 15 minutes
# this just tracks if we need to take a 15 min break or not
j = 1

for item in data_list:
    client.load_cookies('cookies.json')
    handle = item[3]
    user = client.get_user_by_screen_name(handle)
    created = int(str(user.created_at_datetime)[0:4])
    year = int(item[7])
    
    # if the twitter account was created before or during the election year
    # then it's True since we can pull tweets
    if created <= year:
        item[-1] = True
    
    # otherwise we can't pull tweets for this election for this politician
    else:
        item[-1] = False
    
    print(f"Batch {j}, item {i} : {item}")

    # cooldowns because of rate limits
    if i == 90 and j < 3:
        time.sleep(900)
        i = 1
        j += 1
    else:
        time.sleep(1)
        i += 1

# there is probably a better way to do this
# but it works...so I'm not gonna fix it

In [ ]:
import numpy as np

# set numpy INT and BOOL to adapt to SQL acceptable data forms
sqlite3.register_adapter(np.int64, int)
sqlite3.register_adapter(np.bool_, str)

# set up SQL connection and cursor
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

In [ ]:
# create table
c.execute("""CREATE TABLE coordinates (
          key STRING,
          twitter_user_id STRING,
          politician_name STRING,
          twitter_handle STRING,
          x_coordinate INTEGER,
          y_coordinate INTEGER,
          political_party STRING,
          election_year INTEGER,
          country STRING,
          twitter_active_during_election STRING
          )""")

In [ ]:
# ingest the data FINALLY

c.executemany("INSERT INTO coordinates VALUES (?,?,?,?,?,?,?,?,?,?)", data_list)
conn.commit()